In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
import selenium
from selenium import webdriver
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By # 셀레니움 4.0부터 포함된 함수(필수)
import os
import sys
import urllib.request
from urllib import parse
import ssl
from selenium.webdriver.support.ui import Select

In [86]:
# 주류 데이터 불러오기
wine = pd.read_csv('wine400.csv')
mark = pd.read_csv('makgeolli_fin.csv')
brandy = pd.read_csv('brandy_cognac.csv')
whisky = pd.read_csv('whisky_info.csv')
korea = pd.read_csv('korea_list.csv')

In [ ]:
# 각 데이터마다 주종 값 추가 와인, 막걸리, 브랜디/꼬냑, 위스키, 소주, 맥주 
# 칼럼명 통일하기 
# 데이터 합치고 id값 컬럼 추가하기 

In [87]:
wine = wine.replace('wine','와인') # 주종 한글로 변경 

In [95]:
wine.head() # id 주종 중분류 상품명 생산국가 생산자 주요품종 도수 음용온도 추천음식 당도 바디 산도 타닌
mark.head() # 주종 상품명 생산국가 제조사 주원료 용량 도수 
brandy.head() # 주류명 주종 원산지 용량/ml 도수/% 특징 
whisky.head() # 주류명 주종 원산지 용량/ml 도수/% 특징 -> 전처리 필요 \n값 들어가있음 
korea.head() # 주류명 주종 원산지 용량/ml 도수/% 제조사 특성 

,주류명,주종,생산국가,용량,도수,생산자/제조사,특징
0,청하,청주,대한민국,300,13.0,롯데칠성음료,깔끔한 맛과 향을 자랑하는 청하는 1986년 출시 이래 차게 마시는 청주의 대명사로...
1,순희,전통주,대한민국,750,6.0,보해양조,"노령산맥 지하 253m의 천연 암반수로 빚었으며, 16일간의 장기 저온발효 후 숙성..."
2,국순당 생막걸리,전통주,대한민국,750,6.0,국순당,유산균 발효 과정에서 생기는 기분 좋은 산미와 직접 키운 건강한 효모가 만들어내는 ...
3,대박,전통주,대한민국,700,6.0,국순당,"3단 발효법으로 풍미는 높이고 신선함은 그대로 담아냈으며, 냉장숙성을 통해 더욱 부..."
4,옛날 막걸리,전통주,대한민국,750,7.8,국순당,"누룩의 양을 3배 이상 사용하여 맛과 향이 배우 진하고 구수하며, 인공 감미료를 첨..."


In [91]:
mark = mark.drop(['Unnamed: 0'],axis=1)

In [93]:
# 원산지 -> 생산국가로 변경 
brandy = brandy.rename(columns={'원산지':'생산국가'}) # 컬럼명 변경 
whisky = whisky.rename(columns={'원산지':'생산국가'})
korea = korea.rename(columns={'원산지':'생산국가'})
# 상품명 -> 주류명 
wine = wine.rename(columns={'상품명':'주류명'})
mark = mark.rename(columns={'상품명':'주류명'})
# 생산자, 제조사 -> 생산자/제조사 
wine = wine.rename(columns={'생산자':'생산자/제조사'})
mark = mark.rename(columns={'제조사':'생산자/제조사'})
korea = korea.rename(columns={'제조사':'생산자/제조사'})
# 용량 , 도수 통일 
brandy = brandy.rename(columns={'용량/ml':'용량'}) 
whisky = whisky.rename(columns={'용량/ml':'용량'})
korea = korea.rename(columns={'용량/ml':'용량'})
brandy = brandy.rename(columns={'도수/%':'도수'}) 
whisky = whisky.rename(columns={'도수/%':'도수'})
korea = korea.rename(columns={'도수/%':'도수'}) 
#
korea = korea.rename(columns={'특성':'특징'})

In [92]:
# 주요품종 , 주원료 
wine = wine.rename(columns={'주요품종':'포도품종'})

In [96]:
# 주류명 주종 순서 변경 -> 주종 주류명ㅇ으로
brandy = brandy[['주종','주류명','생산국가','용량','도수','특징']]
whisky = whisky[['주종','주류명','생산국가','용량','도수','특징']]
korea = korea[['주종','주류명','생산국가','생산자/제조사','용량','도수','특징']]

In [ ]:
# 위스키 줄바꿈 삭제 특수문제가 아닌 줄바꿈만 삭제하기 위해서 \n 으로 제거 
whisky['용량']= whisky['용량'].str.replace(pat=r'[\n]',repl=r' ',regex=True)
whisky['도수']= whisky['도수'].str.replace(pat=r'[\n]',repl=r' ',regex=True)

In [ ]:
whisky.to_csv('./whisky_info.csv', encoding = 'utf-8-sig',index=False)

In [113]:
# 데이터 합치기 
result = pd.concat([wine,mark,whisky,brandy,korea])

In [117]:
result = result[['주종','중분류','주류명','생산국가','생산자/제조사','포도품종','주원료','용량','도수','음용온도','추천음식','특징','당도','바디','산도','타닌']]
result

,주종,중분류,주류명,생산국가,생산자/제조사,포도품종,주원료,용량,도수,음용온도,추천음식,특징,당도,바디,산도,타닌
0,와인,레드,"7 컬러즈, '그랑 리제르바' 카베르네 소비뇽 뮈스카",칠레,세븐 컬러즈 7 Colores,"카베르네 소비뇽 (Cabernet Sauvignon) 95%, 뮈스까 (Muscat...",NaN,NaN,13.5,16~18 ℃,"채끝 스테이크, 크림 파스타, 블루치즈 버거 등과 잘 어울린다.",NaN,1.0,3.0,3.0,3.0
1,와인,레드,"GCF, 꼬뜨 드 뵈프",프랑스,GCF 그룹 GCF Group,"시라/쉬라즈 (Syrah/Shiraz) , 마르셀란 (Marselan)",NaN,NaN,13.0,16~18 ℃,"뵈프 부르기뇽, 안심 찹 스테이크, 로스트 비프 스테이크 등과 잘 어울린다.",NaN,1.0,3.0,3.0,2.0
2,와인,화이트,"가트, 리슬링",호주,가트 와인즈 Gatt Wines,리슬링 (Riesling) 100%,NaN,NaN,11.5,10~12 ℃,"샐러드, 해산물, 치즈 등과 잘 어울린다.",NaN,2.0,4.0,3.0,1.0
3,와인,레드,갈로 리빙스톤 콩코드,미국,갤로 패밀리 빈야드 Gallo Family Vineyard - E&J Gallo W...,콩코드 (Concord),NaN,NaN,11.0,14~16 ℃,"소시지, 육포, 떡볶이, 순대, 튀김, 족발, 만두, 자장면 등과 잘 어울린다.",NaN,1.0,4.0,3.0,3.0
4,와인,주정강화,글로리아 토니 포트 와인,포르투갈,빈센테 파리아 비뉴스 Vincente Faria VInhos,"틴타 로리즈 (Tinta Roriz) , 투리가 프란카 (Touriga Franca...",NaN,NaN,20.0,14~16 ℃,"리코타, 크림치즈, 말린과일, 케이크, 초콜렛 등 디저트와 잘 어울린다.",NaN,4.0,2.0,4.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
194,과실주,NaN,톡소다,대한민국,무학,NaN,NaN,360,5.0,NaN,NaN,스페인산 화이트와인이 첨가된 스파클링 와인. 톡쏘는 탄산으로 더 청량하게,NaN,NaN,NaN,NaN
195,과실주,NaN,복분자주,대한민국,보해양조,NaN,NaN,375,15.0,NaN,NaN,보해 복분자주는 2004년 출시 이후 보해를 넘어 대한민국을 대표하는 과실주로 국민...,NaN,NaN,NaN,NaN
196,과실주,NaN,복분자주,대한민국,보해양죠,NaN,NaN,750,15.0,NaN,NaN,보해 복분자주는 2004년 출시 이후 보해를 넘어 대한민국을 대표하는 과실주로 국민...,NaN,NaN,NaN,NaN
197,과실주,NaN,매취순,대한민국,보해양조,NaN,NaN,375,14.0,NaN,NaN,‘매취순’은 국내 최대 매실농원인 전남 해남 ‘보해매실농원’에서 직접 수확한 국산 ...,NaN,NaN,NaN,NaN


In [118]:
result.to_csv('./alcohol_df.csv', encoding = 'utf-8-sig')

In [133]:
df = pd.read_csv('alcohol_df.csv')
df

,주종,중분류,주류명,생산국가,생산자/제조사,포도품종,주원료,용량,도수,음용온도,추천음식,특징,당도,바디,산도,타닌
0,와인,레드,"7 컬러즈, '그랑 리제르바' 카베르네 소비뇽 뮈스카",칠레,세븐 컬러즈 7 Colores,"카베르네 소비뇽 (Cabernet Sauvignon) 95%, 뮈스까 (Muscat...",NaN,750,13.5,16~18 ℃,"채끝 스테이크, 크림 파스타, 블루치즈 버거 등과 잘 어울린다.",NaN,1.0,3.0,3.0,3.0
1,와인,레드,"GCF, 꼬뜨 드 뵈프",프랑스,GCF 그룹 GCF Group,"시라/쉬라즈 (Syrah/Shiraz) , 마르셀란 (Marselan)",NaN,750,13,16~18 ℃,"뵈프 부르기뇽, 안심 찹 스테이크, 로스트 비프 스테이크 등과 잘 어울린다.",NaN,1.0,3.0,3.0,2.0
2,와인,화이트,"가트, 리슬링",호주,가트 와인즈 Gatt Wines,리슬링 (Riesling) 100%,NaN,750,11.5,10~12 ℃,"샐러드, 해산물, 치즈 등과 잘 어울린다.",NaN,2.0,4.0,3.0,1.0
3,와인,레드,갈로 리빙스톤 콩코드,미국,갤로 패밀리 빈야드 Gallo Family Vineyard - E&J Gallo W...,콩코드 (Concord),NaN,750,11,14~16 ℃,"소시지, 육포, 떡볶이, 순대, 튀김, 족발, 만두, 자장면 등과 잘 어울린다.",NaN,1.0,4.0,3.0,3.0
4,와인,주정강화,글로리아 토니 포트 와인,포르투갈,빈센테 파리아 비뉴스 Vincente Faria VInhos,"틴타 로리즈 (Tinta Roriz) , 투리가 프란카 (Touriga Franca...",NaN,750,20,14~16 ℃,"리코타, 크림치즈, 말린과일, 케이크, 초콜렛 등 디저트와 잘 어울린다.",NaN,4.0,2.0,4.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2128,과실주,NaN,톡소다,대한민국,무학,NaN,NaN,360,5,NaN,NaN,스페인산 화이트와인이 첨가된 스파클링 와인. 톡쏘는 탄산으로 더 청량하게,NaN,NaN,NaN,NaN
2129,과실주,NaN,복분자주,대한민국,보해양조,NaN,NaN,375,15,NaN,NaN,보해 복분자주는 2004년 출시 이후 보해를 넘어 대한민국을 대표하는 과실주로 국민...,NaN,NaN,NaN,NaN
2130,과실주,NaN,복분자주,대한민국,보해양죠,NaN,NaN,750,15,NaN,NaN,보해 복분자주는 2004년 출시 이후 보해를 넘어 대한민국을 대표하는 과실주로 국민...,NaN,NaN,NaN,NaN
2131,과실주,NaN,매취순,대한민국,보해양조,NaN,NaN,375,14,NaN,NaN,‘매취순’은 국내 최대 매실농원인 전남 해남 ‘보해매실농원’에서 직접 수확한 국산 ...,NaN,NaN,NaN,NaN


In [134]:
df.index=df.index + 1
df.head() # id값 설정 

,주종,중분류,주류명,생산국가,생산자/제조사,포도품종,주원료,용량,도수,음용온도,추천음식,특징,당도,바디,산도,타닌
1,와인,레드,"7 컬러즈, '그랑 리제르바' 카베르네 소비뇽 뮈스카",칠레,세븐 컬러즈 7 Colores,"카베르네 소비뇽 (Cabernet Sauvignon) 95%, 뮈스까 (Muscat...",NaN,750,13.5,16~18 ℃,"채끝 스테이크, 크림 파스타, 블루치즈 버거 등과 잘 어울린다.",NaN,1.0,3.0,3.0,3.0
2,와인,레드,"GCF, 꼬뜨 드 뵈프",프랑스,GCF 그룹 GCF Group,"시라/쉬라즈 (Syrah/Shiraz) , 마르셀란 (Marselan)",NaN,750,13,16~18 ℃,"뵈프 부르기뇽, 안심 찹 스테이크, 로스트 비프 스테이크 등과 잘 어울린다.",NaN,1.0,3.0,3.0,2.0
3,와인,화이트,"가트, 리슬링",호주,가트 와인즈 Gatt Wines,리슬링 (Riesling) 100%,NaN,750,11.5,10~12 ℃,"샐러드, 해산물, 치즈 등과 잘 어울린다.",NaN,2.0,4.0,3.0,1.0
4,와인,레드,갈로 리빙스톤 콩코드,미국,갤로 패밀리 빈야드 Gallo Family Vineyard - E&J Gallo W...,콩코드 (Concord),NaN,750,11,14~16 ℃,"소시지, 육포, 떡볶이, 순대, 튀김, 족발, 만두, 자장면 등과 잘 어울린다.",NaN,1.0,4.0,3.0,3.0
5,와인,주정강화,글로리아 토니 포트 와인,포르투갈,빈센테 파리아 비뉴스 Vincente Faria VInhos,"틴타 로리즈 (Tinta Roriz) , 투리가 프란카 (Touriga Franca...",NaN,750,20,14~16 ℃,"리코타, 크림치즈, 말린과일, 케이크, 초콜렛 등 디저트와 잘 어울린다.",NaN,4.0,2.0,4.0,3.0


In [135]:
df.to_csv('./alcohol_df.csv', encoding = 'utf-8-sig')

In [136]:
df2 = pd.read_csv('alcohol_df.csv')
df2

,Unnamed: 0,주종,중분류,주류명,생산국가,생산자/제조사,포도품종,주원료,용량,도수,음용온도,추천음식,특징,당도,바디,산도,타닌
0,1,와인,레드,"7 컬러즈, '그랑 리제르바' 카베르네 소비뇽 뮈스카",칠레,세븐 컬러즈 7 Colores,"카베르네 소비뇽 (Cabernet Sauvignon) 95%, 뮈스까 (Muscat...",NaN,750,13.5,16~18 ℃,"채끝 스테이크, 크림 파스타, 블루치즈 버거 등과 잘 어울린다.",NaN,1.0,3.0,3.0,3.0
1,2,와인,레드,"GCF, 꼬뜨 드 뵈프",프랑스,GCF 그룹 GCF Group,"시라/쉬라즈 (Syrah/Shiraz) , 마르셀란 (Marselan)",NaN,750,13,16~18 ℃,"뵈프 부르기뇽, 안심 찹 스테이크, 로스트 비프 스테이크 등과 잘 어울린다.",NaN,1.0,3.0,3.0,2.0
2,3,와인,화이트,"가트, 리슬링",호주,가트 와인즈 Gatt Wines,리슬링 (Riesling) 100%,NaN,750,11.5,10~12 ℃,"샐러드, 해산물, 치즈 등과 잘 어울린다.",NaN,2.0,4.0,3.0,1.0
3,4,와인,레드,갈로 리빙스톤 콩코드,미국,갤로 패밀리 빈야드 Gallo Family Vineyard - E&J Gallo W...,콩코드 (Concord),NaN,750,11,14~16 ℃,"소시지, 육포, 떡볶이, 순대, 튀김, 족발, 만두, 자장면 등과 잘 어울린다.",NaN,1.0,4.0,3.0,3.0
4,5,와인,주정강화,글로리아 토니 포트 와인,포르투갈,빈센테 파리아 비뉴스 Vincente Faria VInhos,"틴타 로리즈 (Tinta Roriz) , 투리가 프란카 (Touriga Franca...",NaN,750,20,14~16 ℃,"리코타, 크림치즈, 말린과일, 케이크, 초콜렛 등 디저트와 잘 어울린다.",NaN,4.0,2.0,4.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2128,2129,과실주,NaN,톡소다,대한민국,무학,NaN,NaN,360,5,NaN,NaN,스페인산 화이트와인이 첨가된 스파클링 와인. 톡쏘는 탄산으로 더 청량하게,NaN,NaN,NaN,NaN
2129,2130,과실주,NaN,복분자주,대한민국,보해양조,NaN,NaN,375,15,NaN,NaN,보해 복분자주는 2004년 출시 이후 보해를 넘어 대한민국을 대표하는 과실주로 국민...,NaN,NaN,NaN,NaN
2130,2131,과실주,NaN,복분자주,대한민국,보해양죠,NaN,NaN,750,15,NaN,NaN,보해 복분자주는 2004년 출시 이후 보해를 넘어 대한민국을 대표하는 과실주로 국민...,NaN,NaN,NaN,NaN
2131,2132,과실주,NaN,매취순,대한민국,보해양조,NaN,NaN,375,14,NaN,NaN,‘매취순’은 국내 최대 매실농원인 전남 해남 ‘보해매실농원’에서 직접 수확한 국산 ...,NaN,NaN,NaN,NaN


In [137]:
df2 = df2.rename(columns={'Unnamed: 0':'id'})

In [138]:
df2

,id,주종,중분류,주류명,생산국가,생산자/제조사,포도품종,주원료,용량,도수,음용온도,추천음식,특징,당도,바디,산도,타닌
0,1,와인,레드,"7 컬러즈, '그랑 리제르바' 카베르네 소비뇽 뮈스카",칠레,세븐 컬러즈 7 Colores,"카베르네 소비뇽 (Cabernet Sauvignon) 95%, 뮈스까 (Muscat...",NaN,750,13.5,16~18 ℃,"채끝 스테이크, 크림 파스타, 블루치즈 버거 등과 잘 어울린다.",NaN,1.0,3.0,3.0,3.0
1,2,와인,레드,"GCF, 꼬뜨 드 뵈프",프랑스,GCF 그룹 GCF Group,"시라/쉬라즈 (Syrah/Shiraz) , 마르셀란 (Marselan)",NaN,750,13,16~18 ℃,"뵈프 부르기뇽, 안심 찹 스테이크, 로스트 비프 스테이크 등과 잘 어울린다.",NaN,1.0,3.0,3.0,2.0
2,3,와인,화이트,"가트, 리슬링",호주,가트 와인즈 Gatt Wines,리슬링 (Riesling) 100%,NaN,750,11.5,10~12 ℃,"샐러드, 해산물, 치즈 등과 잘 어울린다.",NaN,2.0,4.0,3.0,1.0
3,4,와인,레드,갈로 리빙스톤 콩코드,미국,갤로 패밀리 빈야드 Gallo Family Vineyard - E&J Gallo W...,콩코드 (Concord),NaN,750,11,14~16 ℃,"소시지, 육포, 떡볶이, 순대, 튀김, 족발, 만두, 자장면 등과 잘 어울린다.",NaN,1.0,4.0,3.0,3.0
4,5,와인,주정강화,글로리아 토니 포트 와인,포르투갈,빈센테 파리아 비뉴스 Vincente Faria VInhos,"틴타 로리즈 (Tinta Roriz) , 투리가 프란카 (Touriga Franca...",NaN,750,20,14~16 ℃,"리코타, 크림치즈, 말린과일, 케이크, 초콜렛 등 디저트와 잘 어울린다.",NaN,4.0,2.0,4.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2128,2129,과실주,NaN,톡소다,대한민국,무학,NaN,NaN,360,5,NaN,NaN,스페인산 화이트와인이 첨가된 스파클링 와인. 톡쏘는 탄산으로 더 청량하게,NaN,NaN,NaN,NaN
2129,2130,과실주,NaN,복분자주,대한민국,보해양조,NaN,NaN,375,15,NaN,NaN,보해 복분자주는 2004년 출시 이후 보해를 넘어 대한민국을 대표하는 과실주로 국민...,NaN,NaN,NaN,NaN
2130,2131,과실주,NaN,복분자주,대한민국,보해양죠,NaN,NaN,750,15,NaN,NaN,보해 복분자주는 2004년 출시 이후 보해를 넘어 대한민국을 대표하는 과실주로 국민...,NaN,NaN,NaN,NaN
2131,2132,과실주,NaN,매취순,대한민국,보해양조,NaN,NaN,375,14,NaN,NaN,‘매취순’은 국내 최대 매실농원인 전남 해남 ‘보해매실농원’에서 직접 수확한 국산 ...,NaN,NaN,NaN,NaN


In [142]:
df2['용량'] = df['용량'].str.replace(r'\D+','')

C:\Users\guddk\AppData\Local\Temp\ipykernel_21480\3129830494.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  df2['용량'] = df['용량'].str.replace(r'\D+','')


In [146]:
df2.to_csv('./alcohol_df.csv', encoding = 'utf-8-sig',index=False)